**Notebook content:**
- Combine all events files into one file.
- Add columns headers.
- Remove empty columns. 
- Remove irrlevant columns. 
- Remove duplicate rows.

In [1]:
%reset -f 

In [4]:
import sys, os
import numpy as np
import pandas as pd
import string
from IPython.display import display
sys.path.append(os.path.dirname(sys.path[0])) #to change to environment 
from utils.constants import *

HEADERS_FILE = "eventsHeader.csv"
EVENTS_FILE = "Events{}.csv"

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

In [2]:
#num files
NUM_FILES = 10

**Get headers from a seperate file** (headers not mentioned - no description). 

**rowId**: unique identifier of the row

**id**: unique identifier of the event in the sysetm

**pid**: annonimized patient identification

**admissionId**: annonimized admission identification

**bValue**: if the record has a bolean value like true or false

**bed**: bed

**bodyLocation**: no description.

**cancelDate**: no description.

**cancelled**: no description.

**converted**: no description.

**dValue**: if the record has a decimal

**eventCode**: code of the event either in snomed, loinc, rxnorm, internal etc.

**eventCodeOrg**: original code in the hospital source systems

**eventDesc**: Description of the event

**eventEndDate**: the date and time that the event ended

**eventName**: additional descriptive field for the event

**eventResult**	NA

**eventResultType**	NA

**eventStartDate**:	date and time that the event started

**eventType**: type of event - which ontology does the eventcode refers to

**eventTypeOrg**: original code family in the hospital source systems

**iValue**: if the record has an integer value

**messageId**:	identifies the message that was recived by the system regarding this event

**orderNumber**: microbiology order number

**organismId**: snomed code of the identified organism in a microbiology test

**parentId**: id of the state that used this event

**sValue**: if the event hase text value

**sourceName**: indicates the source for the event

**unitOfMeasure**: the unit of measure that refers to the values in the record


In [5]:
headers_path = os.path.join(DPATH_SOURCES, HEADERS_FILE) 
with open(headers_path, 'r') as f:
    headers = f.read().strip().split('|')
headers

['rowId',
 'id',
 'pid',
 'admissionId',
 'altAdmissionId',
 'altPid',
 'bValue',
 'abnormalFlags',
 'bed',
 'bodyLocation',
 'cancelDate',
 'cancelled',
 'converted',
 'dValue',
 'eventCode',
 'eventCodeOrg',
 'eventDesc',
 'eventEndDate',
 'eventName',
 'eventResult',
 'eventResultType',
 'eventStartDate',
 'eventType',
 'eventTypeOrg',
 'facility',
 'iValue',
 'messageId',
 'normalRange',
 'orderNumber',
 'organismId',
 'parentId',
 'pointOfCare',
 'presentOnAdmission',
 'principalDiagnosis',
 'room',
 'sValue',
 'sourceName',
 'specimenCollectionSite',
 'specimenId',
 'tValue',
 'transferrable',
 'unitOfMeasure',
 'careGiver',
 'Time_Stamp']

**Explore one Events file.**

Event file has 44 columns. 

In [11]:
file0 = os.path.join(DPATH_SOURCES, EVENTS_FILE.format(0))
df_file0 = pd.read_csv(file0, sep='|', encoding='cp1255', names=headers)
print("Shape:", df_file0.shape)
print()
print(df_file0.info())
print()
display(df_file0.head())

c:\Users\hp4mp\PythonProjects\Anastomotic-Leak-Prediction\Data\sourceFiles\Events0.csv
Shape: (233468, 44)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233468 entries, 0 to 233467
Data columns (total 44 columns):
rowId                     233463 non-null object
id                        233466 non-null object
pid                       233460 non-null float64
admissionId               233460 non-null float64
altAdmissionId            0 non-null float64
altPid                    6 non-null object
bValue                    220083 non-null object
abnormalFlags             0 non-null float64
bed                       178979 non-null object
bodyLocation              0 non-null float64
cancelDate                0 non-null float64
cancelled                 233458 non-null object
converted                 233460 non-null object
dValue                    220083 non-null float64
eventCode                 233460 non-null object
eventCodeOrg              218067 non-null object
eventDesc      

,rowId,id,pid,admissionId,altAdmissionId,altPid,bValue,abnormalFlags,bed,bodyLocation,...,room,sValue,sourceName,specimenCollectionSite,specimenId,tValue,transferrable,unitOfMeasure,careGiver,Time_Stamp
0,63729368,109501336,1.013265e+09,1840641.0,NaN,NaN,False,NaN,NaN,NaN,...,NaN,NaN,[MIRROR_Chameleon].[Lab_Results],NaN,NaN,NaN,True,K/microL,NaN,2018-02-16 06:37:37.807000000
1,63735783,109499667,1.013265e+09,1840641.0,NaN,NaN,False,NaN,NaN,NaN,...,NaN,NaN,[MIRROR_Chameleon].[Lab_Results],NaN,NaN,NaN,True,IU/l,NaN,2018-02-16 06:38:35.467000000
2,63735786,109499668,1.013265e+09,1840641.0,NaN,NaN,False,NaN,NaN,NaN,...,NaN,NaN,[MIRROR_Chameleon].[Lab_Results],NaN,NaN,NaN,True,g/dl,NaN,2018-02-16 06:38:35.467000000
3,63735789,109499669,1.013265e+09,1840641.0,NaN,NaN,False,NaN,NaN,NaN,...,NaN,NaN,[MIRROR_Chameleon].[Lab_Results],NaN,NaN,NaN,True,meq/l,NaN,2018-02-16 06:38:35.467000000
4,63735792,109499670,1.013265e+09,1840641.0,NaN,NaN,False,NaN,NaN,NaN,...,NaN,NaN,[MIRROR_Chameleon].[Lab_Results],NaN,NaN,NaN,True,mg/dl,NaN,2018-02-16 06:38:35.467000000


# Combine files into one dataframe

In [7]:
#path of events files 
paths = [os.path.join(DPATH_SOURCES, EVENTS_FILE.format(i)) for i in range(NUM_FILES)]

In [8]:
#read each file
frames = [pd.read_csv(file, sep='|', encoding='cp1255', names=headers) for file in paths]

C:\Users\hp4mp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0,1,5,11,12,30,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\hp4mp\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0,1,5,11,12,30,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
#get types of each data frame, to verify they can be combined into one file
types_frames = [df.dtypes for df in frames]

**Explore types**

Datatypes of columns are identical along files, 
except of column "tValue" which is of type "float64" in files 1-9, and "object" in file 0

In [10]:
types_df = pd.concat(types_frames, axis=1, ignore_index=True).T
types_df.index = ["file_{}".format(i) for i in range(numFiles)]
types_df

,rowId,id,pid,admissionId,altAdmissionId,altPid,bValue,abnormalFlags,bed,bodyLocation,cancelDate,cancelled,converted,dValue,eventCode,eventCodeOrg,eventDesc,eventEndDate,eventName,eventResult,eventResultType,eventStartDate,eventType,eventTypeOrg,facility,iValue,messageId,normalRange,orderNumber,organismId,parentId,pointOfCare,presentOnAdmission,principalDiagnosis,room,sValue,sourceName,specimenCollectionSite,specimenId,tValue,transferrable,unitOfMeasure,careGiver,Time_Stamp
file_0,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,object,object,object,object,object
file_1,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_2,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_3,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_4,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_5,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_6,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_7,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_8,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object
file_9,object,object,float64,float64,float64,object,object,float64,object,float64,float64,object,object,float64,object,object,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,object,float64,float64,float64,object,object,object,object


In [11]:
t = types_df.nunique(axis=0).ne(1)
print("Colums where types are not the same:", t.loc[t].index.values)

Colums where types are not the same: ['tValue']


The "object" values in column 'tValue', file 0, are dates, while values of column 'tValue' in rest files are null. 

In [12]:
print(df_file0['tValue'].dropna())
df_file1 = frames[1]
print()
print(df_file1['tValue'].dropna())

21414    1900-01-01 09:00:00.0000000
21415    1900-01-01 16:30:00.0000000
Name: tValue, dtype: object

Series([], Name: tValue, dtype: float64)


**Combine events file into one file**

In [13]:
#combine files into one data frame
df = pd.concat(frames, axis=0, ignore_index=True)

In [14]:
df.shape

(1683669, 44)

# Drop rows

**Drop duplicate rows** - There is only one duplicate row.

In [15]:
prev_num_rows = df.shape[0]
df.drop_duplicates(inplace=True)
print("Num duplicate rows:", prev_num_rows - df.shape[0])

Num duplicate rows: 1


**Drop rows where patient id is null** - there are 52 such rows. 

In [16]:
null_pid = df[df["pid"].isna()]
null_pid

,rowId,id,pid,admissionId,altAdmissionId,altPid,bValue,abnormalFlags,bed,bodyLocation,cancelDate,cancelled,converted,dValue,eventCode,eventCodeOrg,eventDesc,eventEndDate,eventName,eventResult,eventResultType,eventStartDate,eventType,eventTypeOrg,facility,iValue,messageId,normalRange,orderNumber,organismId,parentId,pointOfCare,presentOnAdmission,principalDiagnosis,room,sValue,sourceName,specimenCollectionSite,specimenId,tValue,transferrable,unitOfMeasure,careGiver,Time_Stamp
21002,NaN,[MIRROR_Sheba_ICU].[MVQFreeTextSignalsView],nan,nan,nan,True,NaN,nan,2018-03-13 09:36:35.760000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
227989,NaN,[MIRROR_Sheba_ICU].[MVQFreeTextSignalsView],nan,nan,nan,True,NaN,nan,2019-05-18 22:48:12.127000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
231953,93,NaN,nan,nan,nan,NaN,NaN,nan,NaN,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
231954,97%,NaN,nan,nan,nan,NaN,NaN,nan,NaN,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
231955,36.7,[MIRROR_Sheba_ICU].[MVQFreeTextSignalsView],nan,nan,nan,True,NaN,nan,2019-05-21 18:54:57.700000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383799,NaN,[MIRROR_Sheba_ICU].[MVQFreeTextSignalsView],nan,nan,nan,True,NaN,nan,2018-08-14 01:29:10.423000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
1396568,הפעלה,[MIRROR_Chameleon].[Description],nan,nan,nan,True,NaN,nan,2019-04-25 12:15:34.340000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
1467222,נשימתי,[MIRROR_Chameleon].[Description],nan,nan,nan,True,NaN,nan,2018-02-15 18:48:48.417000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN
1492382,NaN,[MIRROR_Sheba_ICU].[MVQFreeTextSignalsView],nan,nan,nan,True,NaN,nan,2018-01-30 16:52:24.220000000,nan,nan,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,NaN,NaN


In [287]:
df = df[df["pid"].notna()]
print("Num rows with missing patient id:", null_pid.shape[0])

Num rows with missing patient id: 52


In [288]:
df.shape[0]

1683616

# Missing values
There are some columns of nan values only. 

In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683616 entries, 0 to 1683668
Data columns (total 44 columns):
rowId                     1683616 non-null object
id                        1683616 non-null object
pid                       1683616 non-null float64
admissionId               1683616 non-null float64
altAdmissionId            0 non-null float64
altPid                    0 non-null object
bValue                    1556931 non-null object
abnormalFlags             0 non-null float64
bed                       1171761 non-null object
bodyLocation              0 non-null float64
cancelDate                0 non-null float64
cancelled                 1683610 non-null object
converted                 1683616 non-null object
dValue                    1556931 non-null float64
eventCode                 1683616 non-null object
eventCodeOrg              1534034 non-null object
eventDesc                 136875 non-null object
eventEndDate              1683616 non-null object
eventName  

**Remove columns of null values only**. 
Num left columns: **29**. There are columns with few non-nan values.  

In [38]:
#drop columns with all NaN's
df = df.dropna(axis=1, how='all')
df.shape

(1683668, 30)

Print percentage of missing values in each column.

In [291]:
# percentage of missing values in each column
percent_missing = df.isna().mean().round(5) * 100
pd.DataFrame({"percent missing values":percent_missing})

,percent missing values
rowId,0.00
id,0.00
pid,0.00
admissionId,0.00
bValue,7.52
bed,30.40
cancelled,0.00
converted,0.00
dValue,7.52
eventCode,0.00


In [292]:
#Remove columns where >99% of values are missing. Num left columns: 26. Num rows: ~1.6M.
'''percent_missing_indexes = percent_missing[percent_missing>99.99].index.tolist()
percent_missing_indexes

df = df.drop(percent_missing_indexes, axis=1)
df.info()
'''

'percent_missing_indexes = percent_missing[percent_missing>99.99].index.tolist()\npercent_missing_indexes\n\ndf = df.drop(percent_missing_indexes, axis=1)\ndf.info()\n'

# Drop columns

Remove irrelevant columns. Num left columns: **18**. Num rows: ~1.6M.

In [296]:
cols_to_remove = ["rowId", "id", "altPid", "bed", "cancelled", "converted", 
                  "messageId", "parentId", "tValue", "transferrable", 
                    "careGiver", "Time_Stamp"]

df.drop(columns=cols_to_remove, inplace=True, errors='ignore')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683616 entries, 0 to 1683668
Data columns (total 18 columns):
pid               1683616 non-null float64
admissionId       1683616 non-null float64
bValue            1556931 non-null object
dValue            1556931 non-null float64
eventCode         1683616 non-null object
eventCodeOrg      1534034 non-null object
eventDesc         136875 non-null object
eventEndDate      1683616 non-null object
eventName         1628158 non-null object
eventStartDate    1683616 non-null object
eventType         1683616 non-null object
eventTypeOrg      1541809 non-null object
iValue            1556931 non-null float64
orderNumber       16699 non-null float64
organismId        2126 non-null float64
sValue            38602 non-null object
sourceName        1683577 non-null object
unitOfMeasure     966464 non-null object
dtypes: float64(6), object(12)
memory usage: 244.1+ MB


# Reorder columns

In [297]:
cols_ordered = ["pid", "admissionId", "eventName",
                "eventStartDate", "eventEndDate", 
                "bValue", "dValue", "iValue", "sValue", 
                "eventDesc", "unitOfMeasure",
                "orderNumber", "organismId",
                "eventCode", "eventCodeOrg", 
                "eventType", "eventTypeOrg", "sourceName"]

df = df[cols_ordered]

# Write output
Write combined events file 

In [298]:
#output into one file 
output_path = "{}parseData0.csv".format(DATA_PATH)
df.to_csv(output_path, sep=',', index=False)